In [1]:
import torch 
import sys
sys.path.append('..')
from torch import nn 
from torch.nn import functional as F
from torch import optim
from utils import dlc_practical_prologue as prologue  
from utils.loader import load

In [2]:
train_input,train_target, train_classes, test_input, test_target, test_classes = load()

mean, std = train_input.mean(), train_input.std()

train_input = train_input.sub_(mean).div_(std)
test_input = test_input.sub_(mean).div_(std)

In [3]:

class LeNet_aux_sequential(nn.Module):
    """
    Weight sharing + Auxiliary loss
    
    """
    def __init__(self):
        super(LeNet_aux_sequential, self).__init__()
        # convolutional weights for digit reocgnition shared for each image
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        
        # weights for binary classification 
        self.fc3 = nn.Linear(20, 60)
        self.fc4 = nn.Linear(60, 90)
        self.fc5 = nn.Linear(90, 2)
        
    def forward(self, input_):    
        
        # split the 2-channel input into two 14*14 images
        x = input_[:, 0, :, :].view(-1, 1, 14, 14)
        y = input_[:, 1, :, :].view(-1, 1, 14, 14)
        
        # forward pass for the first image 
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        
        # forward pass for the second image 
        y = F.relu(F.max_pool2d(self.conv1(y), kernel_size=2, stride=2))
        y = F.relu(F.max_pool2d(self.conv2(y), kernel_size=2, stride=2))
        y = F.relu(self.fc1(y.view(-1, 256)))
        y = self.fc2(y)
        
        # concatenate layers  
        z = torch.cat([x, y], 1)
        
        z = F.relu(self.fc3(z))
        z = F.relu(self.fc4(z))
        z = self.fc5(z)
        
        return x, y, z

In [10]:
##### train function ######

def train_aux (model, train_input, train_target, train_classes, optimizer = optim.SGD,
                criterion = nn.CrossEntropyLoss(), n_epochs=50, mini_batch_size=100, eta=1e-1, lambda_l2 = 0, alpha=0.5, beta=0.5):
    
    
    """
    Train network with auxiliary loss + weight sharing
    
    """

    model.train()
    optimizer = optimizer(model.parameters(), lr = eta)
    
    for e in range(n_epochs):
        epoch_loss = 0
        
        for input_, target_, classes_ in (zip(train_input.split(mini_batch_size), train_target.split(mini_batch_size), 
                                              train_classes.split(mini_batch_size))):

            class_1, class_2, out = model(input_)
            aux_loss1 = criterion(class_1, classes_[:,0])
            aux_loss2 = criterion(class_2, classes_[:,1])
            out_loss  = criterion(out, target_)
            net_loss = (alpha * (out_loss) + beta * (aux_loss1 + aux_loss2) )
            epoch_loss += net_loss
            
            if lambda_l2 != 0:
                for p in model.parameters():
                    epoch_loss += lambda_l2 * p.pow(2).sum() # add an l2 penalty term to the loss 
            
            optimizer.zero_grad()
            net_loss.backward()
            optimizer.step()
            
        print('Train Epoch: {}  | Loss {:.6f}'.format(
                e, epoch_loss.item()))
        
#########################################################################################################################
#########################################################################################################################

### test function  ###

def test_aux(model, test_input, test_target, mini_batch_size=100, criterion = nn.CrossEntropyLoss()):
    
    """
    Test function to calculate prediction accuracy of a cnn with auxiliary loss
    
    """
    
    model.eval()
    test_loss = 0
    nb_errors=0
    
    with torch.no_grad():
        
        for input_, target_ in zip(test_input.split(mini_batch_size), test_target.split(mini_batch_size)):
            
            _, _, output = model(input_) 
            batch_loss = criterion(output, target_)
            test_loss += batch_loss
            
            _, predicted_classes = output.max(1)
            for k in range(mini_batch_size):
                if target_[k] != predicted_classes[k]:
                    nb_errors = nb_errors + 1
                                   
             
        print('\nTest set | Loss: {:.4f} | Accuracy: {:.0f}% | # misclassified : {}/{}\n'.format(
        test_loss.item(), 100 * (len(test_target)-nb_errors)/len(test_target), nb_errors, len(test_target)))
        

In [11]:
model = LeNet_aux_sequential()

train_aux(model, train_input, train_target, train_classes)

test_binary(model, test_input, test_target)

Train Epoch: 0  | Loss 25.975597
Train Epoch: 1  | Loss 24.109648
Train Epoch: 2  | Loss 21.736666
Train Epoch: 3  | Loss 19.104828
Train Epoch: 4  | Loss 13.048098
Train Epoch: 5  | Loss 12.812750
Train Epoch: 6  | Loss 9.317560
Train Epoch: 7  | Loss 7.381166
Train Epoch: 8  | Loss 6.919405
Train Epoch: 9  | Loss 6.043700
Train Epoch: 10  | Loss 5.189643
Train Epoch: 11  | Loss 5.046283
Train Epoch: 12  | Loss 4.149846
Train Epoch: 13  | Loss 3.966151
Train Epoch: 14  | Loss 3.404825
Train Epoch: 15  | Loss 3.176835
Train Epoch: 16  | Loss 2.979700
Train Epoch: 17  | Loss 2.775200
Train Epoch: 18  | Loss 2.606035
Train Epoch: 19  | Loss 2.477594
Train Epoch: 20  | Loss 2.295163
Train Epoch: 21  | Loss 2.197179
Train Epoch: 22  | Loss 2.046921
Train Epoch: 23  | Loss 1.994811
Train Epoch: 24  | Loss 1.945644
Train Epoch: 25  | Loss 1.758901
Train Epoch: 26  | Loss 1.835623
Train Epoch: 27  | Loss 1.641751
Train Epoch: 28  | Loss 1.526894
Train Epoch: 29  | Loss 1.345458
Train Epoch: 3